### Get all author links

In [15]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

home = "http://latin.packhum.org/browse"

author_links = [] # each element: [author name, author link]

def get_links(soup):

    np_link = ''
    
    for tag in soup.findAll('a'): # find tags that contain links
        link = tag['href']
        match = re.findall('/author/[0-9]+', link) # find all links in this format
        if match:
    
            np_link = 'http://latin.packhum.org' + match[0] # create whole link
        
            author_name = tag.findAll('span')
            author_name = author_name[0].text # get author name
        
            if np_link not in author_links:
                author_links.append([author_name, np_link])
                
req = requests.get("http://latin.packhum.org/browse") # the page where we search for author links
soup = BeautifulSoup(req.text, 'lxml')

get_links(soup) # calls function to collect all author links
print(len(author_links))

362


### Get all text links from "author_links"

In [55]:
text_links = [] # each element: [author name, text name, text link]

def get_text_links(soup, author_name):

    np_link = ''
    
    for tag in soup.findAll('a'):
        link = tag['href']
        match = re.findall('/loc/.*$', link) # finds all links to texts
        if match:
        
            text_name = tag.findAll('span')
            text_name = text_name[1].text
            
            print(match[0] + ' ' + author_name + ' ' + text_name)
            
            np_link = 'http://latin.packhum.org' + match[0] # create whole link
            print(np_link)
            
            if np_link not in text_links:
                text_links.append([author_name, text_name, np_link])
                
for i in author_links:
    author_name = i[0]
    author_link = i[1]
    req = requests.get(author_link) # the page where we search for text links
    soup = BeautifulSoup(req.text, 'lxml')
    
    get_text_links(soup, author_name) # calls function to collect all links to texts
    
print(len(text_links))

0


### Get all texts from "text_links"

In [67]:
all_texts = [] # each element: [author, text name, text]

def get_texts(author, text_name, text_link):
    
    req = requests.get(text_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    file_name = author + ' - ' + text_name + '.txt' # unique name for text
    
    f = open(file_name, 'w', encoding = 'utf-8') # creates new file for a text
    
    whole_text = ''
    for tag in soup.findAll('table'):

        for tr in tag.findAll('tr'): # finds all tags with text
            line = tr.text

            while '\n' in line: # deleting empty lines 
                line = line.replace('\n', '')

            while '                           ' in line: # deleting extra spaces
                line = line.replace('                           ', '          ')
            line = line.lstrip() # deleting extra spaces 
            
            whole_text += line
            whole_text += '\n'
            
    f.write(whole_text)          
    f.close()
    
    all_texts.append([author, text_name, whole_text])
    
    print(author + ' - ' + text_name)
    
for item in text_links:
    author = item[0]
    text_name = item[1]
    text_link = item[2]
    
    get_texts(author, text_name, text_link) # creates new file for each text
                                            # the texts are now also in 'all_texts'
    


### Load text from each file, save into "all_texts.txt"

In [66]:
import glob

texts = [] # each element is [author and text name, text]

path = 'text files/'

for filename in glob.glob(os.path.join(path, '*.txt')): # finds all .txt in 'text files' folder

    f = open(filename, 'r', encoding = 'utf-8')
    text = f.read()
    
    texts.append([filename, text])
    
    f.close()
    
print(len(texts))

f1 = open('all_texts.txt', 'w', encoding = 'utf-8') # new file for all texts

for item in texts:
    filename = item[0]
    text = item[1]
    
    f1.write(filename)
    f1.write('\n\n')
    f1.write(text)
    f1.write('-----------------------------------------------')
    f1.write('\n')
    
f1.close()

836
